In [2]:
import datetime
import time
import calendar
from backend.utility import *
user = User.objects.get(email="veyorokon@gmail.com")
invoice = Invoice.objects.filter(user=user)[0]
invoice.save()
print(invoice.user)

Email=veyorokon@gmail.com


In [3]:
member = SubscriptionMember.objects.filter(user=user)[0]
stripeInvoice = invoice.get_stripe_invoice()
print(stripeInvoice)

{
  "amount_due": 6958,
  "amount_paid": 0,
  "amount_remaining": 6958,
  "application_fee": null,
  "attempt_count": 0,
  "attempted": false,
  "billing": "charge_automatically",
  "billing_reason": "upcoming",
  "charge": null,
  "created": 1554102000,
  "currency": "usd",
  "custom_fields": null,
  "customer": "cus_Ee5GDE9rGJ7uW6",
  "date": 1554102000,
  "default_source": null,
  "description": null,
  "discount": null,
  "due_date": null,
  "ending_balance": 0,
  "finalized_at": null,
  "footer": null,
  "lines": {
    "data": [
      {
        "amount": 250,
        "currency": "usd",
        "description": "Anorak Management Fee",
        "discountable": true,
        "id": "ii_1EElmuGjkoMLHlzN1ksuH77U",
        "invoice_item": "ii_1EElmuGjkoMLHlzN1ksuH77U",
        "livemode": false,
        "metadata": {},
        "object": "line_item",
        "period": {
          "end": 1554076800,
          "start": 1552778371
        },
        "plan": null,
        "proration": false,
  

In [15]:
def get_item_from_invoice(plan, invoice):
    stripeInvoice = invoice.get_stripe_invoice()
    item = invoice.find_items(plan.split(), stripeInvoice)[0]
    return item, stripeInvoice

def is_prorated(stripeInvoiceItem):
    itemPrice = stripeInvoiceItem.amount
    planPrice = stripeInvoiceItem.plan.amount
    return itemPrice != planPrice

def days_in_a_month(date):
    return calendar.monthrange(date.year,date.month)[1]

def set_invoice_dates(output, itemStartEpoch, date):
    billingDateTime = convert_epoch(stripeInvoice.period_end)
    
    lastDayOfMonth = days_in_a_month(date)
    endDateTime = datetime.strptime('{0} {1} {2}'.format(date.month, lastDayOfMonth, date.year), '%m %d %Y')
    billingDateTime = convert_epoch(stripeInvoice.period_end)
    endDate = endDateTime.strftime('%B %d, %Y')
    renewalDate = get_first_day_of_next_month(endDateTime).strftime('%B %d, %Y')
    startDate = convert_epoch(itemStartEpoch).strftime('%B %d, %Y')
    billingDate = billingDateTime.strftime('%B %d, %Y')
    
    output["date_end"] = endDate
    output["date_renew"] = renewalDate
    output["date_start"] = startDate
    output["date_billing"] = billingDate
    return output

def get_receipt_data(member, invoice):
    total = 0
    output = {}
    
    plan = member.subscription_account.price_plan.get_product_name()
    item, stripeInvoice = get_item_from_invoice(plan, invoice)
    itemStartEpoch = item.period.start
    if is_prorated(item):
        amount = item.amount/100
        output["items"]=[{"description": item.description,  "price": amount}]
        total += amount
    amount = item.plan.amount/100
    
    output["total"] = (total + amount)
    output["invoice_number"] = stripeInvoice.number
    
    date = convert_epoch(stripeInvoice.period_end)
    billingDateTime = convert_epoch(stripeInvoice.period_end)
    
    output = set_invoice_dates(output, itemStartEpoch, date)
    
    output["lastItem"]={"description": plan+" - Month of "+billingDateTime.strftime('%B'),  "price": amount}
    output["service"]= member.subscription_account.service.name
    return output

def get_invoice_data(invoice):
    stripeInvoice = invoice.get_stripe_invoice()
    invoiceItems = stripeInvoice.lines.data
    print(invoiceItems)
    
def email_finalized_invoice(invoice):
    stripeInvoice = invoice.user.stripe_customer.get_stripe_latest_invoice()
    print(stripeInvoice)

def convert_epoch(epoch):
    date = datetime.fromtimestamp(epoch)
    return date

In [16]:
#data = get_receipt_data(member, invoice)
#print(data)

In [12]:
output = get_invoice_data(invoice)
print(output)

[<InvoiceLineItem line_item id=ii_1EElmuGjkoMLHlzN1ksuH77U at 0x7fb91ab37570> JSON: {
  "amount": 250,
  "currency": "usd",
  "description": "Anorak Management Fee",
  "discountable": true,
  "id": "ii_1EElmuGjkoMLHlzN1ksuH77U",
  "invoice_item": "ii_1EElmuGjkoMLHlzN1ksuH77U",
  "livemode": false,
  "metadata": {},
  "object": "line_item",
  "period": {
    "end": 1554076800,
    "start": 1552778371
  },
  "plan": null,
  "proration": false,
  "quantity": 1,
  "subscription": "sub_Ee5GIgHDyjTmMk",
  "type": "invoiceitem"
}, <InvoiceLineItem line_item id=ii_1EDkdXGjkoMLHlzNMhqrLCM8 at 0x7fb91a949af0> JSON: {
  "amount": 818,
  "currency": "usd",
  "description": "Remaining time on Netflix - Family Plan (4 users) after 14 Mar 2019",
  "discountable": false,
  "id": "ii_1EDkdXGjkoMLHlzNMhqrLCM8",
  "invoice_item": "ii_1EDkdXGjkoMLHlzNMhqrLCM8",
  "livemode": false,
  "metadata": {},
  "object": "line_item",
  "period": {
    "end": 1554102000,
    "start": 1552535622
  },
  "plan": {
    

In [10]:
from mail_templated import EmailMessage

In [11]:
message = EmailMessage('invoice.tpl', 
                       {'user': user, 'data':data}, 
                       'Anorak@ianorak.com', 
                       to =[user.email]
                      )
message.send()

1

In [4]:
from accounting.email import *

In [5]:
user = User.objects.get(email="veyorokon@gmail.com")
invoice = Invoice.objects.filter(user=user)[0]
member = SubscriptionMember.objects.filter(user=user)[0]

In [6]:
email_receipt(member, invoice)

In [17]:
email_finalized_invoice(invoice)

{
  "amount_due": 0,
  "amount_paid": 0,
  "amount_remaining": 0,
  "application_fee": null,
  "attempt_count": 0,
  "attempted": true,
  "auto_advance": false,
  "billing": "charge_automatically",
  "billing_reason": "subscription_create",
  "charge": null,
  "created": 1551830093,
  "currency": "usd",
  "custom_fields": null,
  "customer": "cus_Ee5GDE9rGJ7uW6",
  "date": 1551830093,
  "default_source": null,
  "description": null,
  "discount": null,
  "due_date": null,
  "ending_balance": 0,
  "finalized_at": 1551830093,
  "footer": null,
  "hosted_invoice_url": "https://pay.stripe.com/invoice/invst_B9kp9a4pQHb8AV2Y6u7TEuRhO2",
  "id": "in_1EAn61GjkoMLHlzNaPFAgqpe",
  "invoice_pdf": "https://pay.stripe.com/invoice/invst_B9kp9a4pQHb8AV2Y6u7TEuRhO2/pdf",
  "lines": {
    "data": [
      {
        "amount": 0,
        "currency": "usd",
        "description": "Time on Anorak after 05 Mar 2019",
        "discountable": false,
        "id": "ii_1EAn61GjkoMLHlzNnTNo7VnL",
        "invoice